## Call tactic server with stream

In [7]:
import requests
import json

event_dict = {
    'DraftAgent': '--- DraftAgent ---\n',
    'RefinementAgent_Analysis': '\n\n--- RefinementAgent ---\n[Analysis]: ',
    'RefinementAgent_Translation': '\n[Translation]: ',
    'Final_Translation': '\n[Final Translation]:\n'
}

def request_server_stream(content):
    url = "http://127.0.0.1:20063/v1/chat/completions"
    headers = {"Content-Type": "application/json"}

    data = {
        "model": "deepseek-chat",
        "messages": [
            {
                "role": "system", 
                "content": "You are a helpful assistant."
            },
            {
                "role": "user", 
                "content": content
            }
        ],
        "temperature": 0.6,
        "max_tokens": 2048,
        "stream": True  
    }

    last_type = None  # Track the previous type to avoid repeatedly printing the prefix

    with requests.post(url, headers=headers, data=json.dumps(data), stream=True, timeout=(5, 300)) as response:
        if response.status_code != 200:
            print(f"Error: {response.status_code}")
            print(response.text)
            return

        for line in response.iter_lines(decode_unicode=True):
            if line:
                if line.strip() == "data: [DONE]":
                    print("\n\n[Stream completed]")
                    break
                if line.startswith("data: "):
                    try:
                        payload = json.loads(line[6:])
                        event_type = payload.get("type")
                        delta = payload["choices"][0]["delta"]
                        content = delta.get("content", "")

                        # Determine the type. Only type changes or the prefix is output every time
                        if event_type in event_dict:
                            # If the type changes or is output for the first time
                            if last_type != event_type:
                                print(event_dict[event_type], end='', flush=True)
                                last_type = event_type
                        # Output content
                        print(content, end='', flush=True)
                    except Exception as e:
                        print(f"\n[Parse error]: {e}")
                        print(line)


content = "zh-en:\nHello!"

request_server_stream(content)

--- DraftAgent ---
Hello!

--- RefinementAgent ---
[Analysis]: The candidate translation 'Hello!' is a direct and accurate translation of the source text 'Hello!'. It fully captures the original meaning and tone of the greeting in Chinese.
[Final Translation]:
Hello!

[Stream completed]
